In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import time
import scipy.io as sio
from random import shuffle
import time
from collections import Counter
import sklearn.cluster as cluster
import time
import hdbscan
from os import listdir
import sys
from os.path import isfile, join
import os
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path

# own modules in dir
sys.path.insert(0, '/home/bojk/custom_pymodules')
os.chdir('..')




In [32]:
%pwd

'/home/bojk/Data'

In [37]:
moddf = pd.read_pickle('KEGG_module_list')
KEGGmoddf = pd.read_pickle('KEGG_modules_df')
COGcat = pd.read_table('COGreffiles/fun2003-2014.tab')
COGtab = pd.read_table('COGreffiles/cognames2003-2014.tab')
COGtab['NameNum'] = COGtab['name']+' - '+COGtab['COG']
COGtab['FuncCat'] = [COGcat[COGcat['# Code']==i[0]]['Name'].values[0] for i in COGtab['func']]
COGtab.FuncCat.value_counts()

Function unknown                                                 959
General function prediction only                                 507
Translation, ribosomal structure and biogenesis                  326
Energy production and conversion                                 268
Amino acid transport and metabolism                              265
Carbohydrate transport and metabolism                            239
Inorganic ion transport and metabolism                           210
Cell wall/membrane/envelope biogenesis                           204
Coenzyme transport and metabolism                                204
Transcription                                                    186
Replication, recombination and repair                            176
Posttranslational modification, protein turnover, chaperones     172
Signal transduction mechanisms                                   144
Nucleotide transport and metabolism                              115
Mobilome: prophages, transposons  

# this analyzes the OP aquificae

In [35]:
OPaquificae = [2770939329,2770939370,2770939407]
mypath = 'Obsidian_Pool/aquificae/'
IMGID = [f for f in listdir(mypath) if '.' not in f and 'Knumbers' not in f and 'Analysis' not in f]
ktab = pd.DataFrame(index=range(10000),columns=IMGID)
ptab = pd.DataFrame(index=range(10000),columns=IMGID)
ctab = pd.DataFrame(index=range(10000),columns=IMGID)
ctab_thr = pd.DataFrame(index=range(10000),columns=IMGID)


for bins in IMGID:
    cpath = mypath+bins+'/'+bins+'.cog.tab.txt'
    kpath = mypath+bins+'/'+bins+'.ko.tab.txt'
    ppath = mypath+bins+'/'+bins+'.pfam.tab.txt'
    
    ktab[bins] =  pd.read_csv(kpath,'\t')['ko_id'].str[3:]
    if os.path.isdir(mypath+'Knumbers')==True:
        ktab[bins].to_csv(mypath+'Knumbers/'+bins+'.txt',sep='\t') #write to file for mapper
    else:
        os.mkdir(mypath+'Knumbers')
        ktab[bins].to_csv(mypath+'Knumbers/'+bins+'.txt',sep='\t') #write to file for mapper
           
    pfamtemp =  pd.read_csv(ppath,'\t')[['pfam_id','pfam_name']]
    pfamtemp['namenum'] = pfamtemp['pfam_id']+', '+pfamtemp['pfam_name']
    ptab[bins] = pfamtemp['namenum']
    
    ctab[bins] = pd.read_csv(cpath,'\t')['cog_id']
    ctab_thr[bins] = pd.read_csv(cpath,'\t')['percent_identity']

In [41]:
name = 'Obsidian_Pool/aquificae/Analysis/COG_percent_ID_clade.table'
if isfile(name)==True:
    cmap_thr = pd.read_table(name)
else:
    cmap_thr = pd.DataFrame(index=COGtab.set_index('COG').index,columns=ctab.columns)
    for cols in ctab.columns:
        for cogs in ctab[cols].dropna().unique():
            ind = ctab[cols].dropna()[ctab[cols].dropna().str.contains(cogs)].index.values
            if len(ind)>0:
                val = max(ctab_thr.loc[list(ind),cols])
                cmap_thr.loc[cogs,cols] = val
    cmap_thr.index =  COGtab.set_index('COG').loc[cmap_thr.index]['NameNum']
    cmap_thr.to_csv(name,sep='\t')

In [42]:
def COGmapper(df):
    # gets ctab formatted df into mappable one
    mapdf = pd.DataFrame(index=COGtab.set_index('COG').index,columns=df.columns)
    for names in df.columns:
        mapdf.loc[df[names].dropna(how='any').values,names] = 1
    return(mapdf)

In [43]:
# COG dfs with different percent identity threshold values 
ctab_30 = ctab[ctab_thr>30]
ctab_40 = ctab[ctab_thr>40]
ctab_50 = ctab[ctab_thr>50]
ctab_60 = ctab[ctab_thr>60]
ctab_70 = ctab[ctab_thr>70]
ctab_80 = ctab[ctab_thr>80]

# make heatmap-like dfs 
cmap = COGmapper(ctab)
cmap_30 = COGmapper(ctab_30)
cmap_40 = COGmapper(ctab_40)
cmap_50 = COGmapper(ctab_50)
cmap_60 = COGmapper(ctab_60)
cmap_70 = COGmapper(ctab_70)
cmap_tot = cmap.fillna(0)+cmap_30.fillna(0)+cmap_40.fillna(0)+cmap_50.fillna(0)+cmap_60.fillna(0)+cmap_70.fillna(0)

In [ ]:
"""#def searchGenes(bin_name,keyword):
keyword = 'peptidase'
bin_name = 'Obs4_genome_061'
img_id = lookup_obsidian[lookup_obsidian.index==bin_name]['IMG_ID'].values[0].astype(str)
folder = '/home/datastorage/IMG_ANNOTATION_DATA/Obsidian_MAGs/'+bin_name+'_IMG_'+img_id+'/IMG_Data/'+img_id+'/'        
cofile = folder+img_id+'.cog.tab.txt'
kofile = folder+img_id+'.ko.tab.txt'
pffile = folder+img_id+'.pfam.tab.txt'

codf = pd.read_table(cofile,'\t') #'cog_name','cog_id','percent_identity'
kodf = pd.read_table(kofile,'\t') #'ko_name','ko_id','percent_identity'
kodf = kodf[~kodf['ko_name'].isnull()]
pfdf = pd.read_table(pffile,'\t') #'pfam_name','pfam_id'
    
codf_s = codf[codf['cog_name'].str.contains(keyword)]
kodf_s = kodf[kodf['ko_name'].str.contains(keyword)]
pfdf_s = pfdf[pfdf['pfam_name'].str.contains(keyword)]"""

In [45]:
cmap_thr

,649633104,2512875013,2505679071,643692030,2728369737,2547132489,2551306708,643692050,2724679920,2698536703,...,642555132,642555165,2724679708,2562617012,2770939407,2510461075,649633039,2756170387,2728369220,646311936
NameNum,,,,,,,,,,,,,,,,,,,,,
Glutamate-1-semialdehyde aminotransferase - COG0001,65.51,61.41,55.76,64.57,64.73,60.94,62.15,62.85,63.19,62.41,...,56.47,62.85,64.81,64.02,58.73,55.76,63.66,64.12,63.79,59.76
N-acetyl-gamma-glutamylphosphate reductase - COG0002,53.3,47.26,43.27,47.99,51.73,46.4,46.42,44.38,48.71,48.71,...,43.55,46.7,46.78,45.72,43.84,43.27,48.26,52.89,48.39,47.28
"Anion-transporting ATPase, ArsA/GET3 family - COG0003",31.68,42.15,41.34,NaN,33.77,41.85,NaN,NaN,NaN,39.63,...,41.34,NaN,31.77,NaN,40.85,41.34,33.66,31.25,NaN,40.85
Ammonia channel protein AmtB - COG0004,61.08,60.84,56.79,53.16,35.97,60.84,NaN,57.49,56.65,63.39,...,54.32,58.37,61.33,53.77,60.49,56.79,59.85,NaN,54.01,60.74
Purine nucleoside phosphorylase - COG0005,41.7,50.38,41.83,41.57,42.91,48.86,42.59,43.61,42.51,46.97,...,40.15,42.11,47.49,41.95,43.94,41.83,43.32,42.51,40.45,46.97
Xaa-Pro aminopeptidase - COG0006,37.61,35.12,31.38,33.87,38.08,35.92,34.68,33.33,34.2,33.87,...,32.45,34.95,35.87,33.69,35.12,31.38,36.74,36.07,33.95,35.12
Uroporphyrinogen-III methylase (siroheme synthase) - COG0007,NaN,53.33,NaN,54.47,NaN,NaN,NaN,NaN,NaN,NaN,...,49.79,54.69,NaN,53.88,54.2,NaN,NaN,NaN,53.47,56.25
Glutamyl- or glutaminyl-tRNA synthetase - COG0008,46.78,45.22,43.52,44.7,47.62,45.65,44.38,44.38,43.51,43.76,...,43.28,44.58,45.85,46.15,45.13,43.52,44.28,46.57,45.74,44.92
tRNA A37 threonylcarbamoyladenosine synthetase subunit TsaC/SUA5/YrdC - COG0009,36.46,35.03,29.56,34.18,35.57,35.35,31.05,32.42,34.22,31.82,...,28.92,31.58,31.58,33.16,35.9,29.56,36.08,35.79,33.84,36.1


what to look for in aquificae?
perform KEGG pathway analysis first
look at inter/outersect of COG hits OP vs. clade
core functionality of OP ones?
Which of these genes are not found in clade?

In [46]:
ctab

,649633104,2512875013,2505679071,643692030,2728369737,2547132489,2551306708,643692050,2724679920,2698536703,...,642555132,642555165,2724679708,2562617012,2770939407,2510461075,649633039,2756170387,2728369220,646311936
0,COG2199,COG0593,COG0593,COG3505,COG0486,COG2084,COG0110,COG1672,COG1692,COG1008,...,COG0593,COG0593,COG1112,COG0402,COG1212,COG1940,COG0438,COG0267,COG0402,COG0593
1,COG2200,COG1826,COG0730,COG1484,COG0739,COG0151,COG0399,COG0420,COG0190,COG1007,...,COG0730,COG0566,COG0773,COG2229,COG0486,COG0859,COG0593,COG0690,COG2229,COG1028
2,COG0550,COG0805,COG0073,COG4974,COG1424,COG0018,COG2244,COG0249,COG0755,COG0315,...,COG0073,COG0128,COG0749,COG2018,COG1158,COG4095,COG0592,COG0250,COG2018,COG1692
3,COG0750,COG0287,COG0297,COG4185,COG1490,COG0377,COG0438,COG3016,COG1333,COG1573,...,COG0297,COG0352,COG2206,COG0105,COG1466,COG1242,COG0558,COG0080,COG0105,COG0163
4,COG4974,COG0123,COG0294,COG3451,COG2095,COG0558,COG0573,COG0240,COG0402,COG0707,...,COG0294,COG0265,COG0045,COG0733,COG0131,COG0053,COG0344,COG0081,COG0733,COG0591
5,COG4422,COG0208,COG1624,COG1651,COG0382,COG0735,COG0581,COG2814,COG0617,COG1207,...,COG1624,COG0312,COG0074,COG0373,COG2814,COG0319,COG0142,COG0244,COG0373,COG0436
6,COG1450,COG1324,COG1538,COG4227,COG0647,COG0312,COG1117,COG0705,COG1136,COG2847,...,COG1538,COG2200,COG0674,COG1934,COG1913,COG1702,COG1154,COG0222,COG1934,COG0623
7,COG2805,COG0512,COG1136,COG0550,COG1301,COG2154,COG0704,COG0414,COG0542,COG0597,...,COG1136,COG3547,COG1013,COG3267,COG0165,COG1947,COG1995,COG0085,COG3267,COG1830
8,COG1459,COG0761,COG4591,COG0234,COG0229,COG0312,COG5002,COG0714,COG1385,COG0046,...,COG4591,COG1256,COG1014,COG0128,COG0354,COG0460,COG0108,COG0086,COG0128,COG1763
9,COG2165,COG1489,COG0704,COG2844,COG1353,COG0747,COG0605,COG1305,COG2110,COG0436,...,COG0704,COG1940,COG2050,COG0795,COG5016,COG0849,COG0740,COG0154,COG0795,COG1109


In [5]:
mypath='aquificae/'
os.path.isdir(mypath+'Knumbers')

False